## Bootstrap linear encoding models
**Warning: this notebook is very computationally intensive.** You may wish to run it on your lab server instead of a local machine. Another alternative is to split the subjects up into smaller subsets, then run those subsets instead of holding all the subjects' models in memory at the same time.

In [4]:
from tqdm.notebook import tqdm
import h5py
import pickle
import numpy as np
import random
import itertools as itools
import sys
from glob import glob
sys.path.append('../preprocessing/utils/')
from ridge import eigridge_corr

In [5]:
# Change these path for running the notebook locally
eeg_data_path = '/path/to/dataset/' # downloadable from OSF: https://doi.org/10.17605/OSF.IO/FNRD9
git_path  = '/path/to/git/speaker_induced_suppression_EEG/'
# Where the output of train_linear_model.ipynb is saved. Run that first if you haven't already.
# Alternatively, model pickles are available in the OSF repository, in folder `linear_encoding_model_results`.
pkl_path = '/path/to/pickles/' 
h5_path = '/path/to/h5/'

In [7]:
models = ['model1']
tmin, tmax = -0.3, 0.5
nboots_shuffle = 100
exclude = ['OP0001','OP0002','OP0020']
subjs = np.sort([s[-6:] for s in glob(f'{git_path}eventfiles/*') if 'OP0' in s and s[-6:] not in exclude])
delays = np.arange(np.floor(tmin*128),np.ceil(tmax*128),dtype=int)
chunklen = len(delays)*3 # data randomized in chunks

In [11]:
# Load split data from pickle
for s in tqdm(subjs):
    for m in models:
        # load pickle (strf is stored in h5, vars pre-strf are in a pkl)
        with open(f'{pkl_path}{m}/{s}_B1_tStim.pkl', 'rb') as f:
            tStim = pickle.load(f)
        with open(f'{pkl_path}{m}/{s}_B1_tResp.pkl', 'rb') as f:
            tResp = pickle.load(f)
        with open(f'{pkl_path}{m}/{s}_B1_vStim.pkl', 'rb') as f:
            vStim = pickle.load(f)
        with open(f'{pkl_path}{m}/{s}_B1_vResp.pkl', 'rb') as f:
            vResp = pickle.load(f)
        nsamps, nelecs = tStim.shape
        allinds = range(nsamps)
        nchunks = int(np.floor(0.2*nsamps/chunklen))
        # get alphas from the trained model hdf5
        with h5py.File(f'{h5_path}{m}/{s}_model.hdf5','r') as f:
            alphas = np.array(f.get('alphas'))
        # Run the bootstrap
        corrs = []
        for n in np.arange(nboots_shuffle):
            indchunks = list(zip(*[iter(allinds)]*chunklen))
            random.shuffle(indchunks)
            shuff_inds = list(itools.chain(*indchunks[:nchunks]))
            tStim_shuff = tStim.copy()
            tResp_shuff = tResp.copy()
            tStim_shuff = tStim_shuff[shuff_inds,:]
            tResp_shuff = tResp_shuff[:len(shuff_inds),:]
            corr = eigridge_corr(tStim_shuff, vStim, tResp_shuff, vResp, [alphas[0]], corrmin = 0.05)
            corrs.append(corr)
        boot_corrs=np.vstack((corrs))
        # Compare bootstrap coors to LEM corrs
        # Is the correlation of the model greater than the shuffled correlation for random data?
        with h5py.File(f'{h5_path}{m}/{s}_model.hdf5','r') as f:
            model_corrs = np.array(f.get('corrs'))
        h_val = np.array([model_corrs > boot_corrs[c] for c in np.arange(len(boot_corrs))])
        # Count the number of times out of nboots_shuffle that the correlation is greater than 
        # random, subtract from 1 to get the bootstrapped p_val (one per electrode)
        pvals = 1-h_val.sum(0)/nboots_shuffle
        # Save pvals to the model hdf5
        with h5py.File(f'{h5_path}{m}/{s}_model.hdf5','r+') as f:
            f.create_dataset("pvals", data=pvals)

  0%|          | 0/18 [00:00<?, ?it/s]

Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(32754, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 32754) (32754, 62)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(32754, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 32754) (32754, 62)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(32754, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 32754) (32754, 62)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(32754, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 32754) (32754, 62)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(32754, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 32754) (32754, 62)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(32754, 19)
Covmat shape: 
(19, 19)
(19, 19) (19,

Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(32754, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 32754) (32754, 62)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(32754, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 32754) (32754, 62)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(32754, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 32754) (32754, 62)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(32754, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 32754) (32754, 62)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(32754, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 32754) (32754, 62)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(32754, 19)
Covmat shape: 
(19, 19)
(19, 19) (19,

Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(41097, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 41097) (41097, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(41097, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 41097) (41097, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(41097, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 41097) (41097, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(41097, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 41097) (41097, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(41097, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 41097) (41097, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(41097, 19)
Covmat shape: 
(19, 19)
(19, 19) (19,

Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(41097, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 41097) (41097, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(41097, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 41097) (41097, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(41097, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 41097) (41097, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(41097, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 41097) (41097, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(41097, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 41097) (41097, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(41097, 19)
Covmat shape: 
(19, 19)
(19, 19) (19,

...

Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(34299, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 34299) (34299, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(34299, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 34299) (34299, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(34299, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 34299) (34299, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(34299, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 34299) (34299, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(34299, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 34299) (34299, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(34299, 19)
Covmat shape: 
(19, 19)
(19, 19) (19,

Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(28737, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 28737) (28737, 63)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(28737, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 28737) (28737, 63)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(28737, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 28737) (28737, 63)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(28737, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 28737) (28737, 63)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(28737, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 28737) (28737, 63)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(28737, 19)
Covmat shape: 
(19, 19)
(19, 19) (19,

Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(28737, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 28737) (28737, 63)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(28737, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 28737) (28737, 63)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(28737, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 28737) (28737, 63)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(28737, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 28737) (28737, 63)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(28737, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 28737) (28737, 63)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(28737, 19)
Covmat shape: 
(19, 19)
(19, 19) (19,

Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(36771, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 36771) (36771, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(36771, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 36771) (36771, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(36771, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 36771) (36771, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(36771, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 36771) (36771, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(36771, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 36771) (36771, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(36771, 19)
Covmat shape: 
(19, 19)
(19, 19) (19,

Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(36771, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 36771) (36771, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(36771, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 36771) (36771, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(36771, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 36771) (36771, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(36771, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 36771) (36771, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(36771, 19)
Covmat shape: 
(19, 19)
(19, 19) (19, 36771) (36771, 64)
Cmode = False
Number of time points is greater than the number of features
Rstim shape (not cmode): 
(36771, 19)
Covmat shape: 
(19, 19)
(19, 19) (19,